# Failed attempt to make an ice mask!

All of the below failed to simply upsample a monthly array to daily... see `PROJECTS/scale-aware-air-sea/paper/ice_mask_brute_force.ipynb` for an truly unelegant (but wayyy fast)
solution

In [1]:
# ! pip install "rechunker>0.5.0" # install rechunker from main to get the fix for this https://github.com/pangeo-data/rechunker/issues/135
# this does not work?

In [2]:
# !pip install -e /home/jovyan/PROJECTS/scale-aware-air-sea

In [3]:
from scale_aware_air_sea.utils import open_zarr
# from scale_aware_air_sea.cm26_utils import load_and_combine_cm26
from scale_aware_air_sea.parameters import get_params
import numpy as np
import gcsfs
import matplotlib.pyplot as plt
import zarr
from rechunker import rechunk

import cftime
import xarray as xr
from zarr.storage import FSStore

In [4]:
import rechunker
rechunker.__version__

'0.5.1'

# Literally does not even start with a gateway cluster....urrrhhhhhh

In [5]:
# import dask
# from dask_gateway import Gateway
# gateway = Gateway()

# # close existing clusters
# open_clusters = gateway.list_clusters()
# print(list(open_clusters))
# if len(open_clusters)>0:
#     for c in open_clusters:
#         cluster = gateway.connect(c.name)
#         cluster.shutdown()  

# options = gateway.cluster_options()
# options = gateway.cluster_options()
# options.worker_memory = 16

# # Create a cluster with those options
# cluster = gateway.new_cluster(options)
# client = cluster.get_client()
# cluster.scale(100)
# client

In [6]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=4, threads_per_worker=4)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jbusecke/proxy/8787/status,
Dashboard: /user/jbusecke/proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 125.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42985,Workers: 4
Dashboard: /user/jbusecke/proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 125.81 GiB
Comm: tcp://127.0.0.1:36281,Total threads: 4
Dashboard: /user/jbusecke/proxy/40477/status,Memory: 31.45 GiB
Nanny: tcp://127.0.0.1:40703,


In [16]:
# load global parameters
params = get_params('v0.7.0', test=False) 

fs = gcsfs.GCSFileSystem()

mapper_online_flux = fs.get_mapper('gs://cmip6/GFDL_CM2_6/control/ocean_boundary')
mapper_offline_flux = fs.get_mapper(params['paths']['CM26']['filter_fluxes'])
# these are not defined anymore
# mapper_ice_temp = fs.get_mapper(params['paths']['CM26']['ice_mask_temp'])
# mapper_ice_rechunked = fs.get_mapper(params['paths']['CM26']['ice_mask_rechunked'])
# mapper_ice_interpolated = fs.get_mapper(params['paths']['CM26']['ice_mask_interpolated'])
mapper_ice = fs.get_mapper(params['paths']['CM26']['ice_mask'])

ds_online = open_zarr(mapper_online_flux)
ds_offline = open_zarr(mapper_offline_flux)

In [17]:
# wipe everythign and start over
for mapper in [mapper_ice, mapper_ice_interpolated, mapper_ice_rechunked, mapper_ice_temp]:
    fs.rm(mapper.root, recursive=True)

KeyboardInterrupt: 

## Pad the edges of the melt array

In [8]:
# creating a time resolved ice mask
melt = ds_online.melt

melt_padded = melt.pad(pad_width={'time':1}, mode='edge')
# replace the time with properly padded values
padded_time = np.hstack([
    np.array([cftime.DatetimeJulian(180, 12, 15, 12, 0, 0)]),
    melt.time.data,
    np.array([cftime.DatetimeJulian(201, 1, 16, 12, 0, 0)])
])
melt_padded = melt_padded.assign_coords(time=padded_time)
melt_padded

<xarray.DataArray 'melt' (time: 242, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<concatenate, shape=(242, 2700, 3600), dtype=float32, chunksize=(1, 2700, 3600), chunktype=numpy.ndarray>
Coordinates:
    geolat_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 450), meta=np.ndarray>
    geolon_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 450), meta=np.ndarray>
  * time      (time) object 0180-12-15 12:00:00 ... 0201-01-16 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Attributes:
    cell_methods:   time: mean
    long_name:      water flux transferred with sea ice form/melt (>0 enters ...
    standard_name:  water_flux_into_sea_water_due_to_sea_ice_thermodynamics
    time_avg_info:  average_T1,average_T2,average_DT
    units:          (kg/m^3)*(m/sec)
    valid_range:    [-1000000.0, 1000000.0]

## Rechunk

In [9]:
# melt_padded.chunk({'time':242, 'xt_ocean':5})

In [22]:
rechunked = rechunk(
    melt_padded.to_dataset(name='melt'),
    target_chunks={'time':242, 'xt_ocean':5}, #might have to increase this and up the rechunk memory to reduce tasks (currently ~300k)
    target_store=FSStore('gs://'+mapper_ice_rechunked.root),
    temp_store=FSStore('gs://'+mapper_ice_temp.root),
    max_mem="4GB"
)
rechunked

<Rechunked>
* Source      : <xarray.Dataset>
Dimensions:   (yt_ocean: 2700, xt_ocean: 3600, time: 242)
Coordinates:
    geolat_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 450), meta=np.ndarray>
    geolon_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 450), meta=np.ndarray>
  * time      (time) object 0180-12-15 12:00:00 ... 0201-01-16 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    melt      (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 2700, 3600), meta=np.ndarray>

* Intermediate: <zarr.hierarchy.Group '/'>

* Target      : <zarr.hierarchy.Group '/'>

In [23]:
rechunked.execute()

<zarr.hierarchy.Group '/'>

In [24]:
# delete temp store (to reuse below)
fs.rm(mapper_ice_temp.root, recursive=True)

## interpolate the time

In [25]:
melt_reloaded = xr.open_dataset(mapper_ice_rechunked, engine='zarr', chunks={})
melt_interpolated = melt_reloaded.interp(time=ds_offline.time)
melt_interpolated

/tmp/ipykernel_2401/290783814.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  melt_reloaded = xr.open_dataset(mapper_ice_rechunked, engine='zarr', chunks={})


<xarray.Dataset>
Dimensions:   (yt_ocean: 2700, xt_ocean: 3600, time: 7305)
Coordinates:
    geolat_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 5), meta=np.ndarray>
    geolon_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 5), meta=np.ndarray>
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * time      (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
Data variables:
    melt      (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(7305, 2700, 5), meta=np.ndarray>

In [26]:
# write out *AGAIN*
melt_interpolated.to_zarr(mapper_ice_interpolated)

## rechunk back to the old shape..wohooo

In [10]:
melt_interpolated_reloaded = xr.open_dataset(mapper_ice_interpolated, engine='zarr', chunks={})
melt_interpolated_reloaded

<xarray.Dataset>
Dimensions:   (yt_ocean: 2700, xt_ocean: 3600, time: 7305)
Coordinates:
    geolat_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 5), meta=np.ndarray>
    geolon_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 5), meta=np.ndarray>
  * time      (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    melt      (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(7305, 2700, 5), meta=np.ndarray>

In [13]:
# # REMOVE Reset the stores in case something goes wrong and I need to restart (remove in final version)
fs.rm(mapper_ice_temp.root, recursive=True)
fs.rm(mapper_ice.root, recursive=True)

In [14]:
rechunk_plan_reverse = rechunk(
    melt_interpolated_reloaded.isel(time=slice(0,3*100)),
    target_chunks={'time':3, 'xt_ocean':3600},
    target_store=FSStore('gs://'+mapper_ice.root),
    temp_store=FSStore('gs://'+mapper_ice_temp.root),
    # max_mem="4GB", # local Cluster config
    max_mem = "1GB", # for the default gateway workers
    # If I have to do this again, I should probably ask for more mem on the workers? Or just get even more workers
)
rechunk_plan_reverse

<Rechunked>
* Source      : <xarray.Dataset>
Dimensions:   (yt_ocean: 2700, xt_ocean: 3600, time: 300)
Coordinates:
    geolat_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 5), meta=np.ndarray>
    geolon_t  (yt_ocean, xt_ocean) float32 dask.array<chunksize=(338, 5), meta=np.ndarray>
  * time      (time) object 0181-01-01 12:00:00 ... 0181-10-27 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    melt      (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(300, 2700, 5), meta=np.ndarray>

* Intermediate: <zarr.hierarchy.Group '/'>

* Target      : <zarr.hierarchy.Group '/'>

In [15]:
rechunk_plan_reverse.execute()

<zarr.hierarchy.Group '/'>

In [31]:
import zarr
zarr.consolidate_metadata(mapper_ice)


KeyboardInterrupt



In [ ]:
xr.open_dataset(mapper_ice, engine='zarr', chunks={})

In [13]:
# # delete all temp/intermediate stores
# fs.rm(mapper_ice_temp.root,  recursive=True)
# fs.rm(mapper_ice_rechunked.root,  recursive=True)
# fs.rm(mapper_ice_interpolated.root,  recursive=True)
# #dont do this unless something went wrong fs.rm(mapper_ice.root,  recursive=True)